<p align="center">
    <img src="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/raw/main/images/drosofig.png" height="350" alt="skeletonization image">
</p>

# Skeleton analysis of a drosophila trachea
---

In this notebook, we will study the elongated branching structure of a drosophila trachea in a 3D confocal image. We will introduce vesselness filters, skeletonization and show how the connected filaments structure can be represented as a graph of nodes and vertices using the [Skan](https://skeleton-analysis.org/stable/) package in Python.

### Acknowledgements

We kindly acknowledge [Lemaitre lab](https://www.epfl.ch/labs/lemaitrelab/) in EPFL for providing the data for this notebook!

### Setup

Check that you have all the necessary packages installed, including `napari` and `Skan`. If not, you can use the `!` symbol to install them directly from the Jupyter notebook (otherwise, you can use your terminal).

In [1]:
!pip install skan

In [2]:
import napari
from skan import Skeleton, summarize

### Get the data

The image we'll use in this tutorial is available for download on [Zenodo](https://zenodo.org/record/8099852) (`drosophila_trachea.tif`).

In the cell below, we use a Python package called [pooch](https://pypi.org/project/pooch/) to automatically download the image from Zenodo into the **data** folder of this repository.

In [3]:
import pooch
from pathlib import Path

data_path = Path('.').resolve().parent / 'data'
fname = 'drosophila_trachea.tif'

pooch.retrieve(
    url="https://zenodo.org/record/8099852/files/drosophila_trachea.tif",
    known_hash="md5:d595ac271779936e255afd0508cca43f",
    path=data_path,
    fname=fname,
    progressbar=True,
)

print(f'Downloaded image {fname} into: {data_path}')

Downloaded image drosophila_trachea.tif into: /home/wittwer/code/eias-2023-visualization-workshop/data


### Read the image

We use the `imread` function from Scikit-image to read our TIF image.

In [4]:
from skimage.io import imread

image = imread(data_path / 'drosophila_trachea.tif')

print(f'Loaded image in an array of shape: {image.shape} and data type {image.dtype}')
print(f'Intensity range: [{image.min()} - {image.max()}]')

Loaded image in an array of shape: (30, 250, 250) and data type uint8
Intensity range: [0 - 255]


If you run into troubles, don't hesitate to ask for help 🤚🏽.

### Load the image into Napari

Let's open a viewer and load our image to have a look at it.

In [5]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(image, name="Original image")

../src/intel/isl/isl.c:2220: FINISHME: ../src/intel/isl/isl.c:isl_surf_supports_ccs: CCS for 3D textures is disabled, but a workaround is available.


<Image layer 'Original image' at 0x7fe10fdcd910>

### Intensity normalization

Let's rescale our image to the range 0-1. By doing so, it is also converted to an array of data type `float`.

In [6]:
from skimage.exposure import rescale_intensity

image_normed = rescale_intensity(image, out_range=(0, 1))

print(f'Intensity range: [{image_normed.min()} - {image_normed.max()}]')
print(f'Array type: {image_normed.dtype}')

Intensity range: [0.0 - 1.0]
Array type: float64


### Vesselness filtering

[Vesselness filters](https://scikit-image.org/docs/stable/auto_examples/edges/plot_ridge_filter.html#sphx-glr-auto-examples-edges-plot-ridge-filter-py) are designed to highlight elongated structures. Here, we apply the *meijering* filter from Scikit-image to our normalized image.

In [7]:
from skimage.filters import meijering

image_mei = meijering(image_normed, sigmas=range(1, 3, 1), black_ridges=False)

# Have a look at what the filtered image looks like in Napari:
viewer.add_image(image_mei, name="Vesselness filter", contrast_limits=(0, 1));

### Thresholding

Next, we apply [Otsu's method](https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html) to binarize the image.

In [8]:
from skimage.filters import threshold_otsu
import numpy as np

labels_binary = (image_mei >= threshold_otsu(image_mei)).astype(np.uint8)

# Have a look at the result
viewer.add_labels(labels_binary, name="Otsu threshold");

### Skeletonization

Finally, we use Scikit-image to [skeletonize](https://scikit-image.org/docs/stable/auto_examples/edges/plot_skeleton.html) the image to extract a binary trace at the center of the filaments.

In [9]:
from skimage.morphology import skeletonize_3d

img_skeleton = skeletonize_3d(labels_binary)

viewer.add_image(
    img_skeleton, 
    name="Skeleton", 
    contrast_limits=(0, 1), 
    colormap='green', 
    blending='additive'
)

<Image layer 'Skeleton' at 0x7fe106ee6a30>

### Analysis of the skeleton

Using [skan.Skeleton](https://skeleton-analysis.org/stable/), we extract a **graph representation of the skeleton** in the form of a [Pandas DataFrame](https://pandas.pydata.org/).

In [10]:
table = summarize(Skeleton(skeleton_image=img_skeleton))

# Let's remove isolated branches (indexed as branch-type zero) from the table.
table.drop(table[table['branch-type'] == 0].index, axis=0, inplace=True)

table.head()

,skeleton-id,node-id-src,node-id-dst,branch-distance,branch-type,mean-pixel-value,stdev-pixel-value,image-coord-src-0,image-coord-src-1,image-coord-src-2,image-coord-dst-0,image-coord-dst-1,image-coord-dst-2,coord-src-0,coord-src-1,coord-src-2,coord-dst-0,coord-dst-1,coord-dst-2,euclidean-distance
7,10,15,1161,5.878315,1,1.0,0.0,0,12,188,3,10,192,0,12,188,3,10,192,5.385165
13,22,43,1509,4.414214,1,1.0,0.0,0,30,125,4,30,126,0,30,125,4,30,126,4.123106
17,30,66,370,9.560478,1,1.0,0.0,0,38,146,1,39,138,0,38,146,1,39,138,8.124038
18,30,68,370,5.414214,1,1.0,0.0,0,39,133,1,39,138,0,39,133,1,39,138,5.099020
22,27,80,1542,4.414214,1,1.0,0.0,0,49,130,4,48,130,0,49,130,4,48,130,4.123106


### Visualization in Napari

There is a little bit of extra work involved to prepare a `vectors` array of shape (N, 2, 3) - the format expected by Napari - where each row encodes the origin and end points of a 3D vector.

In [11]:
import pandas as pd

# Prepare the vectors for Napari
points_src = table[['image-coord-src-0', 'image-coord-src-1', 'image-coord-src-2']].values
points_dst = table[['image-coord-dst-0', 'image-coord-dst-1', 'image-coord-dst-2']].values

directions = points_dst - points_src

vectors = np.concatenate((points_src[np.newaxis], directions[np.newaxis]), axis=0)
vectors = np.swapaxes(vectors, 0, 1)

vectors.shape

(830, 2, 3)

We can finally display our `vectors` in the viewer!

In [12]:
# Add a `Vectors` layer
viewer.add_vectors(vectors, name="Branches", edge_width=0.6, 
    features=pd.DataFrame({'branch-type': table['branch-type'].values.astype(float)}),
    edge_color='branch-type',
)

<Vectors layer 'Branches' at 0x7fe07409ef10>

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<p align="center">
    <a href="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/blob/main/examples/README.md">🔙 Back to case studies</a>
</p>
